In [11]:
import os
import glob
import numpy as np
import cv2
import joblib
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
dsSaveDir = 'npyData'
modelSaveDir = 'models'

In [12]:
try:
    # Load images and labels if they have already been created
    splits = X_train, X_test, X_valid, y_train, y_test, y_valid = [
        np.load(dsSaveDir + '/{}.npy'.format(i+1)) for i in range(6)]

except:
    def glob_images(dir):
        return glob.glob(os.path.abspath(dir + '/*'))

    def read_images(files):
        return [cv2.cvtColor(cv2.imread(file), cv2.COLOR_BGR2RGB) for file in files]

    # Glob images' filepaths
    splits = [glob_images(dir) for dir in ['crutch/train', 'crutch/test', 'crutch/valid',
                                           'wheelchair/train', 'wheelchair/test', 'wheelchair/valid']]

    # Read images
    splits = [read_images(split) for split in splits]

    # Resize crutch images to 416x416 to match wheelchair images
    splits[:3] = [[cv2.resize(img, (416, 416)) for img in split]
                  for split in splits[:3]]

    # Concatenate wheelchair and crutch images
    X_train, X_test, X_valid = [np.concatenate(
        (splits[i], splits[i+3])) for i in range(3)]

    # Create labels, 0 for wheelchair, 1 for crutch
    y_train, y_test, y_valid = [np.concatenate(
        (np.ones(len(splits[i])), np.zeros(len(splits[i+3])))) for i in range(3)]

    # Save images and labels
    splits = [X_train, X_test, X_valid, y_train, y_test, y_valid]
    if not os.path.exists(dsSaveDir):
        os.makedirs(dsSaveDir)
    for i, split in enumerate(splits):
        np.save(dsSaveDir + '/{}'.format(i+1), split)

In [13]:
# Reshape for some models into 2D array
XRe_train, XRe_test, XRe_valid = [
    X.reshape(X.shape[0], -1) for X in splits[:3]]

In [14]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression

try:
    # Load model if already created
    model = joblib.load(modelSaveDir + '/logisticRegression.joblib')
except:
    # Create model
    model = LogisticRegression(max_iter=200, n_jobs=-1)

    # Fit model using training data and validation data
    model.fit(XRe_train, y_train)

    # Save model
    if not os.path.exists(modelSaveDir):
        os.makedirs(modelSaveDir)
    joblib.dump(model, modelSaveDir + '/logisticRegression.joblib')

# Use validation data also for testing since sklearn doesn't support validation during training
XRe_test = np.concatenate((XRe_test, XRe_valid))
y_test = np.concatenate((y_test, y_valid))

# Make predictions
y_pred = model.predict(XRe_test)

# Print metrics
print('No. of iterations to converge: {}'.format(model.n_iter_[0]))
print('\nClassification Report:\n', classification_report(y_test, y_pred))
print('Confusion Matrix:\n', np.array2string(confusion_matrix(y_test, y_pred, normalize='true'),
                                             precision=2, separator=' ', suppress_small=True).replace('\n', '\n '))
print('\nAccuracy: {:.2f}'.format(accuracy_score(y_test, y_pred)))

No. of iterations to converge: 153

Classification Report:
               precision    recall  f1-score   support

         0.0       0.93      0.94      0.93       174
         1.0       0.95      0.94      0.94       207

    accuracy                           0.94       381
   macro avg       0.94      0.94      0.94       381
weighted avg       0.94      0.94      0.94       381

Confusion Matrix:
 [[0.94 0.06]
  [0.06 0.94]]

Accuracy: 0.94
